In [ ]:
import numpy as np
import json as js

In [ ]:
class Layer:
    def __init__(self, input_size, output_size, activation, weights=None, bias=None):
        if weights is None:
            self.weights = np.random.randn(output_size, input_size)
        else:
            self.weights = np.array(weights)
        
        if bias is None:
            self.bias = np.random.randn(output_size, 1)
        else:
            self.bias = bias
            
        self.activation = activation

        self.initial_weights = self.weights.copy()
        self.initial_bias = self.bias.copy()

    
    def activate(self, x):
        if self.activation == 'linear':
            return self.linear(x)
        elif self.activation == 'relu':
            return self.relu(x)
        elif self.activation == 'sigmoid':
            return self.sigmoid(x)
        elif self.activation == 'softmax':
            return self.softmax(x)
    
    def linear(self, x):
        return x
    
    def relu(self, x):
        return np.maximum(0, x)
    
    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))
    
    def softmax(self, x):
        exps = np.exp(x - np.max(x))
        return exps / np.sum(exps, axis=0)
    
    def forward_propagation(self, x):
        return self.activate(np.dot(self.weights, x) + self.bias)


In [ ]:
class FFNN:
    def __init__(self):
        self.layers = []
    
    def add_layer(self, input_size, output_size, activation, weights=None, bias=None):
        if weights is None:
            self.layers.append(Layer(input_size, output_size, activation))
        else:
            self.layers.append(Layer(input_size, output_size, activation, weights, bias))
    
    def forward_propagation(self, x):
        for layer in self.layers:
            x = layer.forward_propagation(x)
        return x


In [ ]:
# def read_json(filename):
#     with open(filename, 'r') as file:
#         data = json.load(file)
    
#     model_specs = data["case"]["model"]
#     input_data = np.array(data["case"]["input"])
    
#     # Extract weights for each layer
#     weights = []
#     for layer_spec in model_specs["layers"]:
#         layer_weights = np.array(layer_spec.get("weights", np.random.randn(layer_spec["number_of_neurons"], model_specs["input_size"])))
#         weights.append(layer_weights)

#     expected_output = np.array(data["expect"]["output"])
    
#     return model_specs, input_data, weights, expected_output

# read json using model
json_file = "linear.json"
with open (json_file, "r") as file:
    data = js.load(file)

print(data)


# # Load JSON data
# model_specs, input_data, weights, expected_output = read_json('linear.json')

# # Initialize FFNN model
# ffnn = FFNN()
# input_size = model_specs["input_size"]
# for layer_spec, layer_weights in zip(model_specs["layers"], weights):
#     number_of_neurons = layer_spec["number_of_neurons"]
#     activation_function = layer_spec["activation_function"]
#     ffnn.add_layer(input_size, number_of_neurons, activation_function, weights=layer_weights)
#     input_size = number_of_neurons

# # Perform forward propagation
# outputs = []
# for instance in input_data:
#     output = ffnn.forward_propagation(instance.reshape(-1, 1))
#     outputs.append(output.flatten())

# # Compare outputs with expected outputs
# for output, expected in zip(outputs, expected_output):
#     print(f"Output: {output}, Expected: {expected}, Match: {np.allclose(output, expected)}")


